In [ ]:
import os

from astropy.time import Time, TimeDelta
from bokeh.models import DatetimeTickFormatter
import holoviews as hv
from holoviews import opts
hv.extension('bokeh', logo=False)
from IPython.display import Markdown as md
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import yaml

from lsst_efd_client import EfdClient

%matplotlib inline

In [ ]:
ifilename = "m1m3_hardpointdata.yaml"
efd_name = "ncsa_teststand_efd"
start_time_str = "2020-09-21T18:30:00"
end_time_str = "2020-09-21T20:30:00"

In [ ]:
with open(ifilename) as ifile:
    info = yaml.safe_load(ifile)
try:
    csc_index = info["cscIndex"]
except KeyError:
    csc_index = None
try:
    info['expectedRate'] = eval(info['expectedRate'])
except TypeError:
    pass

In [ ]:
client = EfdClient(efd_name)

In [ ]:
start_time = Time(start_time_str, scale='utc')
end_time = Time(end_time_str, scale='utc')

columns = ["private_sndStamp", "private_rcvStamp", "private_seqNum"]
df = await client.select_time_series(info['tablename'],
                                     columns,
                                     start_time,
                                     end_time,
                                     index=csc_index)

In [ ]:
md(f"# <center>EFD Diagnostic Report<br><br>{info['plotTitle']}</center>")

In [ ]:
md(f"## <center>{start_time} - {end_time}</center>")

In [ ]:
md("### Breaks in Sequence Number")

In [ ]:
# Look for breaks in sequence numbers
seq_nums = df['private_seqNum'].values
seq_delta = seq_nums[1:] - seq_nums[:-1]
indexes = np.where(seq_delta > 1)[0]
if indexes.size:
    print("Break Size\tStart Timestamp\t\t\t\tEnd Timestamp")
    for index in indexes:
        #print(index)
        print(f"{seq_delta[index]}\t\t{df.index[index]}\t{df.index[index + 1]}")
        #print(df.index[index + 1] - df.index[index])
else:
    print("No breaks in sequence number found.")

In [ ]:
md("### Timing Information")

In [ ]:
d = df["private_sndStamp"].values
delta2 = d[1:] - d[:-1]
rate = 1 / info['expectedRate']
import astropy.units as u
message_send_jitter = (np.median(delta2) - rate) * u.s
if message_send_jitter < (1 * u.s):
    message_send_jitter = message_send_jitter.to(u.ms)
print(f"Median Message Send Jitter: {message_send_jitter:.4f}")

In [ ]:
plt.style.use("./plot_style.mplstyle")
min_t = np.round(np.min(delta2), decimals=3)
max_t = np.round(np.max(delta2), decimals=3)
nbins = 50
fig = plt.figure(1, (14, 6))
fig.suptitle("Distribution of $\Delta$T between Send Timestamps")
ax1 = fig.gca()
x = ax1.hist(delta2, bins=50, log=True)
ax1.set_xlabel("$\Delta$Snd (s)")
h = ax1.set_ylabel("Frequency")

In [ ]:
df["clock_diff"] = df["private_rcvStamp"] - df["private_sndStamp"]

In [ ]:
dt_format = '%F %T'
formats = {'days': dt_format, 'months': dt_format, 'hours': dt_format, 'minutes': dt_format}
date_formatter = DatetimeTickFormatter(**formats)
tick_rotation = 75

table = hv.Table(df.reset_index())
x_tuple = ('index', 'clock_diff')
clock_diff = hv.Curve(table, x_tuple, ("clock_diff")).opts(xlabel="Time", ylabel="Rcv - Snd (s)")
layout = clock_diff
layout.opts(opts.Curve(height=400, width=800, xformatter=date_formatter, xrotation=tick_rotation,
                       padding=0.01))

In [ ]:
plt.style.use("./plot_style.mplstyle")
max_t = np.round(np.max(df["clock_diff"].values), decimals=3)
min_t = np.round(np.min(df["clock_diff"].values), decimals=3)
if not min_t < 0:
    min_t = 0.
nbins = 50
fig = plt.figure(1, (14, 6))
fig.suptitle("Distribution of Receive - Send Timestamps")
ax1 = fig.gca()
x = ax1.hist(df["clock_diff"].values, np.linspace(min_t, max_t, nbins), log=True)
ax1.set_xlabel("Rcv - Snd (s)")
h = ax1.set_ylabel("Frequency")

In [ ]:
# Bokeh histogram plot with logy doesn't seem to work. Leave this cell here
# in case there is time to figure out why.
# Using log conversion doesn't work either as values are zero which represents
# 1 counts don't appear in the plot.
#hv.extension('matplotlib', logo=False)
#max_t = np.round(np.max(df["clock_diff"].values), decimals=3)
#nbins = 50
#hist = np.histogram(df["clock_diff"].values, np.linspace(0, max_t, nbins))
#log_hist = np.log(hist[0])
#log_hist = np.nan_to_num(log_hist, neginf=-1)
#print(hist)

#delta_t = hv.Histogram((hist[1], hist[0]), kdims=hv.Dimension('time differences', 
#                        label='Rcv - Snd', unit='s'),
#                       vdims=hv.Dimension('frequency', label='Log Frequency'),
#                       label='Distribution of 𝚫T')
#delta_t.opts(opts.Figure(logy=True), opts.Histogram(height=400, width=800, padding=0.01))